In [4]:
# noexport

import os
os.system('export_notebook reconstruct_spans_persecond.ipynb')

0

In [2]:
from tmilib import *

from reconstruct_focus_times_common import *
from sorted_collection import SortedCollection
from rescuetime_utils import *

import sklearn
import sklearn.svm
import sklearn.linear_model
import sklearn.ensemble
import sklearn.naive_bayes
import sklearn.tree

from math import log

import datetime
import time

import cPickle as pickle

In [ ]:
import sys

In [3]:
training_users = get_training_users()
test_users = get_test_users()

In [4]:
#print domain_to_id('www.facebook.com')
#print id_to_domain(29708)

In [5]:


def extract_secondlevel_dataset_from_users(users):
  all_labels = []
  all_sinceprev = []
  all_tonext = []
  all_fromdomain = []
  all_todomain = []
  for user in users:
    data = extract_secondlevel_dataset_from_user(user)
    all_labels.extend(data['label'])
    all_sinceprev.extend(data['sinceprev'])
    all_tonext.extend(data['tonext'])
    all_fromdomain.extend(data['fromdomain'])
    all_todomain.extend(data['todomain'])
  return {
    'label': all_labels,
    'sinceprev': all_sinceprev,
    'tonext': all_tonext,
    'fromdomain': all_fromdomain,
    'todomain': all_todomain,
  }

@jsonmemoized
def extract_secondlevel_dataset_for_training():
  return extract_secondlevel_dataset_from_users(training_users)

@jsonmemoized
def extract_secondlevel_dataset_for_test():
  return extract_secondlevel_dataset_from_users(test_users)


In [6]:
def extract_tensecondlevel_dataset_from_users(users):
  all_labels = []
  all_sinceprev = []
  all_tonext = []
  all_fromdomain = []
  all_todomain = []
  for user in users:
    data = extract_secondlevel_dataset_from_user(user, True)
    all_labels.extend(data['label'])
    all_sinceprev.extend(data['sinceprev'])
    all_tonext.extend(data['tonext'])
    all_fromdomain.extend(data['fromdomain'])
    all_todomain.extend(data['todomain'])
  return {
    'label': all_labels,
    'sinceprev': all_sinceprev,
    'tonext': all_tonext,
    'fromdomain': all_fromdomain,
    'todomain': all_todomain,
  }

@jsonmemoized
def extract_tensecondlevel_dataset_for_training():
  return extract_tensecondlevel_dataset_from_users(training_users)

@jsonmemoized
def extract_tensecondlevel_dataset_for_test():
  return extract_tensecondlevel_dataset_from_users(test_users)


In [7]:
#a=extract_secondlevel_training_data_from_user(training_users[0])
#print 'extracting dataset for training'
#extract_secondlevel_dataset_for_training()
#extract_tensecondlevel_dataset_for_training()
#print 'extracting dataset for test'
#extract_secondlevel_dataset_for_test()
#extract_tensecondlevel_dataset_for_test()
#print 'extraction done'

In [8]:
#print zipkeys({'a':[3,4,5], 'b':[6,7,8]}, 'a', 'b')

In [9]:
'''
@memoized
def total_usage_of_domains_in_training():
  return sum_values_in_list_of_dict([get_domain_to_time_spent_for_user(user)for user in training_users])

@memoized
def top_n_domains_by_usage(n=10):
  domain_to_usage = total_usage_of_domains_in_training()
  return [x[0] for x in sorted(domain_to_usage.items(), key=itemgetter(1), reverse=True)[:n]]
'''


'\n@memoized\ndef total_usage_of_domains_in_training():\n  return sum_values_in_list_of_dict([get_domain_to_time_spent_for_user(user)for user in training_users])\n\n@memoized\ndef top_n_domains_by_usage(n=10):\n  domain_to_usage = total_usage_of_domains_in_training()\n  return [x[0] for x in sorted(domain_to_usage.items(), key=itemgetter(1), reverse=True)[:n]]\n'

In [10]:
#print top_n_domains_by_visits()

In [11]:
#print top_n_domains_by_usage(10)
#print top_n_domains_by_usage(5)

In [12]:
#print domain_to_id('newtab')

In [13]:
'''
def dataset_to_feature_vectors(dataset):
  topdomains = numpy.array([domain_to_id(x) for x in top_n_domains_by_visits(20)])
  num_features = 3 + 2*len(topdomains) + 2*len(get_rescuetime_productivity_levels())
  output = [[0]*num_features for x in xrange(len(dataset['sinceprev']))]
  #output = numpy.zeros((len(dataset['sinceprev']), num_features), dtype=object) # object instead of float, so we can have floats and ints
  for idx,sinceprev,tonext,fromdomain,todomain in zipkeys_idx(dataset, 'sinceprev', 'tonext', 'fromdomain', 'todomain'):
    cur = output[idx]
    cur[0] = sinceprev
    cur[1] = tonext
    cur[2] = int(fromdomain == todomain)
    feature_num = 3
    for domain_idx,domain in enumerate(topdomains):
      cur[feature_num+domain_idx] = int(fromdomain == domain)
    feature_num += len(topdomains)
    for domain_idx,domain in enumerate(topdomains):
      cur[feature_num+domain_idx] = int(todomain == domain)
    feature_num += len(topdomains)
    fromdomain_name = id_to_domain(fromdomain)
    todomain_name = id_to_domain(todomain)
    fromdomain_productivity = domain_to_productivity(fromdomain_name)
    todomain_productivity = domain_to_productivity(todomain_name)
    for productivity_idx,productivity in enumerate(get_rescuetime_productivity_levels()):
      cur[feature_num+productivity_idx] = int(fromdomain_productivity == productivity)
    feature_num += len(get_rescuetime_productivity_levels())
    for productivity_idx,productivity in enumerate(get_rescuetime_productivity_levels()):
      cur[feature_num+productivity_idx] = int(todomain_productivity == productivity)
    feature_num += len(get_rescuetime_productivity_levels())
  return output
'''

def remove_cached_features():
  os.remove('get_test_feature_vector.msgpack')
  os.remove('get_training_feature_vector.msgpack')
  #os.remove('get_test_labels.msgpack')
  #os.remove('get_training_labels.msgpack')


In [14]:
#remove_cached_features()

In [15]:
'''
@msgpackmemoized
def get_training_feature_vector():
  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_training())

@msgpackmemoized
def get_training_labels():
  return extract_tensecondlevel_dataset_for_training()['label']

@msgpackmemoized
def get_test_feature_vector():
  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_test())

@msgpackmemoized
def get_test_labels():
  return extract_tensecondlevel_dataset_for_test()['label']
'''

"\n@msgpackmemoized\ndef get_training_feature_vector():\n  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_training())\n\n@msgpackmemoized\ndef get_training_labels():\n  return extract_tensecondlevel_dataset_for_training()['label']\n\n@msgpackmemoized\ndef get_test_feature_vector():\n  return dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_test())\n\n@msgpackmemoized\ndef get_test_labels():\n  return extract_tensecondlevel_dataset_for_test()['label']\n"

In [16]:
'''
def train_classifier_on_data(training_data):
  #classifier = sklearn.tree.DecisionTreeClassifier(max_depth=2) # .71 on test
  #classifier = sklearn.tree.DecisionTreeClassifier(max_depth=1)
  #classifier = sklearn.tree.DecisionTreeClassifier()
  #classifier = sklearn.naive_bayes.GaussianNB() # .73 on test
  #classifier = sklearn.svm.LinearSVC()
  #classifier = sklearn.linear_model.SGDClassifier(class_weight='balanced')
  classifier = sklearn.linear_model.SGDClassifier(loss='modified_huber') # .73 on test
  classifier.fit(dataset_to_feature_vectors(training_data), numpy.array(training_data['label']))
  return classifier

def get_classifier():
  return train_classifier_on_data(extract_tensecondlevel_dataset_for_training())
'''

#global_feature_filter = '11100000000000000000000000000000000000000000000000000'
global_feature_filter = '1'*53

def get_feature_filter():
  return global_feature_filter
  #return '11000000000000000000000000000000000000000000000000000'
  #return '11111111111111111111111111111111111111111111111111111'
  #return '11100000000000000000000000000000000000000000000000000'
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
  #selected_features = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
  #selected_features = [True,False,True,False,False,False,False,True,True,False,True,False]
  return ''.join(map(str, selected_features))

'''
def get_filtered_features():
  selected_features = get_feature_filter()
  selected_features_str = ''.join(map(str, selected_features))
  selected_features_filename = 'filtered_features_' + selected_features_str + '.msgpack'
  if path.exists(selected_features_filename):
    #return numpy.loadtxt()
    #return json.load(open(selected_features_filename))
    return msgpack.load(open(selected_features_filename))
  output = filter_features(get_training_feature_vector())
  msgpack.dump(output, open(selected_features_filename, 'w'))
  #json.dump(output, open(selected_features_filename, 'w'))
  return output
'''

'''
def get_filtered_features_train():
  selected_features = get_feature_filter()
  selected_features_str = ''.join(map(str, selected_features))
  selected_features_filename = 'features_train_' + selected_features_str + '.msgpack'
  if path.exists(selected_features_filename):
    return msgpack.load(open(selected_features_filename))
  output = dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_training(), selected_features)
  msgpack.dump(output, open(selected_features_filename, 'w'))
  return output

def get_filtered_features_test():
  selected_features = get_feature_filter()
  selected_features_str = ''.join(map(str, selected_features))
  selected_features_filename = 'features_test_' + selected_features_str + '.msgpack'
  if path.exists(selected_features_filename):
    return msgpack.load(open(selected_features_filename))
  output = dataset_to_feature_vectors(extract_tensecondlevel_dataset_for_test(), selected_features)
  msgpack.dump(output, open(selected_features_filename, 'w'))
  return output
'''

is_second = len(sys.argv) > 1 and sys.argv[1] == 'second'
print 'is_second: ' + str(is_second)

@memoized
def get_filtered_features_train():
  selected_features = get_feature_filter()
  if is_second:
    return numpy.array(get_feature_vector_for_secondlevel_train(selected_features))
  return numpy.array(get_feature_vector_for_tensecondlevel_train(selected_features))

@memoized
def get_filtered_features_test():
  selected_features = get_feature_filter()
  if is_second:
    return numpy.array(get_feature_vector_for_secondlevel_test(selected_features))
  return numpy.array(get_feature_vector_for_tensecondlevel_test(selected_features))

@memoized
def get_test_labels():
  if is_second:
    return numpy.array(get_labels_for_secondlevel_test())
  return numpy.array(get_labels_for_tensecondlevel_test())

@memoized
def get_training_labels():
  if is_second:
    return numpy.array(get_labels_for_secondlevel_train())
  return numpy.array(get_labels_for_tensecondlevel_train())

#def get_labels_for_user(user):
#  return get_tensecondlevel_activespan_labels_for_user(user)

'''
# we normally want to use the tensecond level ones
def get_filtered_features_train():
  selected_features = get_feature_filter()
  return get_feature_vector_for_secondlevel_train(selected_features)

def get_filtered_features_test():
  selected_features = get_feature_filter()
  return get_feature_vector_for_secondlevel_test(selected_features)

def get_test_labels():
  return get_labels_for_secondlevel_test()

def get_training_labels():
  return get_labels_for_secondlevel_train()

def get_labels_for_user(user):
  return get_secondlevel_activespan_labels_for_user(user)
'''


'''
def filter_features(arr):
  # from get_selected_features()
  selected_features = get_feature_filter()
  #selected_feature_idx = [i for i,x in enumerate(selected_features) if x]
  #return arr[:,selected_feature_idx]
  output = []
  for line in arr:
    output.append([line[i] for i,x in enumerate(selected_features) if x])
    #output.append([x for i,x in enumerate(line) if selected_features[i]])
  return output
'''

#classifier_algorithm = sklearn.ensemble.AdaBoostClassifier

def get_classifier():
  #classifier = sklearn.naive_bayes.GaussianNB()
  #classifier = sklearn.linear_model.SGDClassifier(loss='modified_huber') # .73 on test
  #classifier = sklearn.linear_model.SGDClassifier()
  #classifier = sklearn.ensemble.RandomForestClassifier()
  #classifier = sklearn.ensemble.AdaBoostClassifier()
  #classifier = sklearn.ensemble.GradientBoostingClassifier()
  classifier = classifier_algorithm()
  classifier.fit(get_filtered_features_train(), get_training_labels())
  return classifier

feature_vector_for_tensecondlevel_insession_train/11100000000000000000000000000000000000000000000000000.json
tensecondlevel_activespan_dataset_insession_for_user/g34wuNJgSm.json
insession_seconds_for_user/g34wuNJgSm.json
tensecondlevel_activespan_dataset_insession_for_user/qM6L9Z5Ss9.json
insession_seconds_for_user/qM6L9Z5Ss9.json
tensecondlevel_activespan_dataset_insession_for_user/8QkC8G5H2H.json
insession_seconds_for_user/8QkC8G5H2H.json
tensecondlevel_activespan_dataset_insession_for_user/qsSxPV131T.json
insession_seconds_for_user/qsSxPV131T.json
tensecondlevel_activespan_dataset_insession_for_user/3aWG01QnPf.json
insession_seconds_for_user/3aWG01QnPf.json
tensecondlevel_activespan_dataset_insession_for_user/xtOJXxtOqe.json
insession_seconds_for_user/xtOJXxtOqe.json
tensecondlevel_activespan_dataset_insession_for_user/apytwOyBle.json
insession_seconds_for_user/apytwOyBle.json
tensecondlevel_activespan_dataset_insession_for_user/fx8NZArV8y.json
insession_seconds_for_user/fx8NZArV8y.

In [18]:
#a=get_filtered_features_test()

In [19]:
#print len(a)

In [20]:
#b=get_test_labels()

In [21]:
#print len(b)

In [22]:
#print len(get_feature_filter())

In [23]:
#print len(get_training_feature_vector()[0])

In [24]:
'''
def make_predictions_with_classifier_on_dataset(classifier, dataset):
  return classifier.predict(dataset_to_feature_vectors(dataset))

def make_proba_predictions_with_classifier_on_dataset(classifier, dataset):
  return [x[1] for x in classifier.predict_proba(dataset_to_feature_vectors(dataset))]

def evaluate_classifier(classifier):
  test_predictions = make_predictions_with_classifier_on_dataset(classifier, test_data)
  print sklearn.metrics.classification_report(test_data['label'], test_predictions)
'''

def make_predictions_with_classifier_on_test(classifier):
  #return classifier.predict(filter_features(numpy.array(get_test_feature_vector())))
  return classifier.predict(get_filtered_features_test())

def evaluate_classifier(classifier):
  test_predictions = make_predictions_with_classifier_on_test(classifier)
  print sklearn.metrics.classification_report(get_test_labels(), test_predictions)

def evaluate_classifier_for_user(classifier, user):
  dataset = extract_secondlevel_dataset_from_user(user, True)
  test_labels = get_labels_for_user(user)
  

In [25]:
#a= get_training_feature_vector()

In [26]:
def get_selected_features_rfe():
  classifier = sklearn.linear_model.SGDClassifier()
  selector = sklearn.feature_selection.RFE(classifier, 10, step=1)
  selector = selector.fit(numpy.array(get_training_feature_vector()), numpy.array(get_training_labels()))
  return {
    'n_features': selector.n_features_,
    'support': map(int, selector.support_),
    'ranking': map(int, selector.ranking_),
  }
  # return selector.ranking_

def get_selected_features_rfecv():
  classifier = sklearn.linear_model.SGDClassifier()
  selector = sklearn.feature_selection.RFECV(classifier, step=1)
  selector = selector.fit(numpy.array(get_training_feature_vector()), numpy.array(get_training_labels()))
  return {
    'n_features': selector.n_features_,
    'support': map(int, selector.support_),
    'ranking': map(int, selector.ranking_),
  }

def get_selected_features_chi2():
  selector = sklearn.feature_selection.chi2(numpy.array(get_training_feature_vector()), numpy.array(get_training_labels()))
  return {
    'chi2': selector[0],
    'pval': selector[1],
  }

In [27]:
'''
pickle_file = 'classifier_threefeatures_randomforest.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

"\npickle_file = 'classifier_threefeatures_randomforest.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

In [28]:
if is_second:
  global_feature_filter = '1'*53
  #classifier_algorithm = sklearn.ensemble.RandomForestClassifier

  pickle_file = 'classifier_allfeatures_randomforest_second_v5.pickle'
  if path.exists(pickle_file):
    classifier = pickle.load(open(pickle_file))
  else:
    classifier = get_classifier()
    pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

  evaluate_classifier(classifier)
else:
  for criterion,min_samples_split,min_samples_leaf in shuffled(list(itertools.product(
    ['gini', 'entropy'],
    [1, 2, 3, 4],
    [1, 2, 3],
  ))):
    features_string = '_'.join(map(str, ['criterion', criterion, 'min_samples_split', min_samples_split, 'min_samples_leaf', min_samples_leaf]))
    if is_second:
      model_path = sdir_path('scikit_randomforest_allfeatures_second_v5_' + features_string + '.pickle')
    else:
      model_path = sdir_path('scikit_randomforest_allfeatures_tensecond_v5_' + features_string + '.pickle')
    if path.exists(model_path):
      continue
    try:
      print model_path
      classifier_algorithm = lambda: sklearn.ensemble.RandomForestClassifier(criterion=criterion, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
      classifier = get_classifier()
      pickle.dump(classifier, open(model_path, 'w'), pickle.HIGHEST_PROTOCOL)
      evaluate_classifier(classifier)
    except:
      traceback.print_exc()
      continue

             precision    recall  f1-score   support

      False       0.83      0.81      0.82   1295568
       True       0.83      0.84      0.83   1383871

avg / total       0.83      0.83      0.83   2679439



feature_vector_for_tensecondlevel_insession_train/11111111111111111111111111111111111111111111111111111.json
feature_vector_for_tensecondlevel_insession_test/11111111111111111111111111111111111111111111111111111.json
             precision    recall  f1-score   support

      False       0.84      0.85      0.85   1295568
       True       0.86      0.85      0.85   1383871

avg / total       0.85      0.85      0.85   2679439



In [30]:
'''
pickle_file = 'classifier_threefeatures_randomforest_v2.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

"\npickle_file = 'classifier_threefeatures_randomforest_v2.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

In [31]:
'''
pickle_file = 'classifier_allfeatures_randomforest.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

"\npickle_file = 'classifier_allfeatures_randomforest.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

In [32]:
'''
pickle_file = 'classifier_allfeatures_randomforest_v2.pickle'
if path.exists(pickle_file):
  classifier = pickle.load(open(pickle_file))
else:
  classifier = get_classifier()
  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)

evaluate_classifier(classifier)
'''

"\npickle_file = 'classifier_allfeatures_randomforest_v2.pickle'\nif path.exists(pickle_file):\n  classifier = pickle.load(open(pickle_file))\nelse:\n  classifier = get_classifier()\n  pickle.dump(classifier, open(pickle_file, 'w'), pickle.HIGHEST_PROTOCOL)\n\nevaluate_classifier(classifier)\n"

In [33]:
#training_features = get_training_feature_vector()

In [34]:
#for line in training_features:
#  if line[4] == 1:
#    print line
#    break

In [35]:
#dataset = extract_secondlevel_dataset_from_user(training_users[0], True)

In [36]:
#feature_vectors = dataset_to_feature_vectors(dataset)

In [37]:
#for line in feature_vectors:
#  if line[5] == 1:
#    print line
#    break

In [38]:
#fromdomain_set = set(dataset['fromdomain'])

In [39]:
#print domain_to_id(top_n_domains_by_visits()[0])

In [40]:
#print [x for x in fromdomain_set if id_to_domain(x) == 'www.mturk.com']

In [41]:
'''
print 'get_selected_features_chi2'
print datetime.datetime.fromtimestamp(time.time())
print get_selected_features_chi2()
print 'get_selected_features_rfe'
print datetime.datetime.fromtimestamp(time.time())
print get_selected_features_rfe()
print 'get_selected_features_rfecv'
print datetime.datetime.fromtimestamp(time.time())
print get_selected_features_rfecv()
#print get_selected_features_rfe()
'''

"\nprint 'get_selected_features_chi2'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_chi2()\nprint 'get_selected_features_rfe'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_rfe()\nprint 'get_selected_features_rfecv'\nprint datetime.datetime.fromtimestamp(time.time())\nprint get_selected_features_rfecv()\n#print get_selected_features_rfe()\n"

In [42]:
#print selector.support_
#print selector.ranking_

In [43]:
#classifier = get_classifier()
#evaluate_classifier(classifier)
#evaluate_classifier(get_classifier())
#classifier = get_classifier()
#test_predictions = make_predictions_with_classifier_on_test(classifier)
'''
precision_all,recall_all,_ = sklearn.metrics.precision_recall_curve(get_test_labels(), test_predictions)

best_f1 = 0.0
precision = 0.0
recall = 0.0
for x,y in zip(precision_all,recall_all):
  f1 = 2*(x*y)/(x+y)
  if f1 > best_f1:
    best_f1 = f1
    precision = x
    recall = y
print best_f1, precision, recall
'''

'\nprecision_all,recall_all,_ = sklearn.metrics.precision_recall_curve(get_test_labels(), test_predictions)\n\nbest_f1 = 0.0\nprecision = 0.0\nrecall = 0.0\nfor x,y in zip(precision_all,recall_all):\n  f1 = 2*(x*y)/(x+y)\n  if f1 > best_f1:\n    best_f1 = f1\n    precision = x\n    recall = y\nprint best_f1, precision, recall\n'